# Project I - Finding the Higgs Boson

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas
import math

from helper_functions import *
from proj1_helpers import *
from implementations import *

## Loading and pre-processing the data

In [2]:
# Loading of datasets

DATA_TRAIN = '../data/train.csv'
y, tx, ids = load_csv_data(DATA_TRAIN)

DATA_TEST = '../data/test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST)

In [3]:
# Gestion of the features
#!!!!!!!!Il faut enlever les -999 avant de standardiser!!!!!!
tx[tx ==-999] = np.nan
tx_clean = nan_handling(tx,value=-1,coef=0.7)
tx, _, _ = standardize(tx_clean)
#tx = feature_handling(tx)

## Machine Learning Techniques Testing

In [28]:
def estimate_error(w, testX, testY):
    """ridge regression demo."""
    y_pred = predict_labels(w,testX)
    y_err = testY - y_pred
    error = float(y_err.sum(axis = 0))/testX.shape[0]
    
    return 1-abs(error)

In [4]:
# Splitting the training dataset
ratio = 0.7
seed = 4
trainX, testX, trainY, testY = split_data(tx, y, ratio, seed)

### Least Squares

In [29]:
w, mse = least_squares(trainY, trainX)
error = estimate_error(w, testX, testY)
error

0.72584

### Least Square GD

In [43]:
initial_w = np.ones(30)
max_iters = 50
gamma = 0.2
w, mse = least_squares_GD(y, tx, initial_w, max_iters, gamma)
error = estimate_error(w, testX, testY)
error

0.92336

### Least Squares SGD

In [40]:
initial_w = np.ones(30)
max_iters = 50
gamma = 0.2
w, loss = least_squares_SGD(y, tx, initial_w, max_iters, gamma)
error = estimate_error(w, testX, testY)
error

0.9525066666666666

### Logistic Regression

In [44]:
initial_w = np.ones(30)
max_iters = 50
gamma = 0.2
w = logistic_regression(y, tx, initial_w, max_iters, gamma)
error = estimate_error(w, testX, testY)
error

helper_functions.py:131: RuntimeWarning: divide by zero encountered in log
  loss = np.transpose(y).dot(np.log(pred)) + np.transpose(1-y).dot(np.log(1-pred))


IndexError: invalid index to scalar variable.

### Ridge Regression

In [ ]:
def ridge_regression_demo(x, y, degree, ratio, seed):
    """ridge regression demo."""
    # define parameter
    lambdas = np.logspace(-5, 0, 15)

    trainX, testX, trainY, testY = split_data(x, y, ratio, seed)

    train_matrix = build_poly(trainX, degree)
    test_matrix = build_poly(testX, degree)

    rmse_tr = []
    rmse_te = []
    for ind, lambda_ in enumerate(lambdas):

        w, _ = ridge_regression(trainY, train_matrix, lambda_)
        train_pred = train_matrix.dot(w)
        test_pred = test_matrix.dot(w)
        
        train_error = train_pred - trainY
        test_error = test_pred - testY
        
        rmse_tr.append(math.sqrt(2*(train_error**2).mean()))
        rmse_te.append(math.sqrt(2*(test_error**2).mean()))
        
        print("proportion={p}, degree={d}, lambda={l:.3f}, Training RMSE={tr:.3f}, Testing RMSE={te:.3f}".format(
               p=ratio, d=degree, l=lambda_, tr=rmse_tr[ind], te=rmse_te[ind]))
        
    # Plot the obtained results
    plot_train_test(rmse_tr, rmse_te, lambdas, degree)

In [ ]:
# For the moment, we separate our dataset between a training and a testing set. To Delete later
ratio = 0.7

seed = 56
degree = 3
split_ratio = 0.7
ridge_regression_demo(tx, y, degree, split_ratio, seed)

## Results preparation

### Ridge Regression

In [ ]:
def ridge_regression_final(trainX, trainY, testX, degree, lambda_):
    """ridge regression demo."""

    train_matrix = build_poly(trainX, degree)
    test_matrix = build_poly(testX, degree)

    w, _ = ridge_regression(trainY, train_matrix, lambda_)
    print w
    y_pred = predict_labels(w, testX)
    return y_pred

In [ ]:
# Visualizations
degree = 7
lambda_ = 0.1

train_matrix = build_poly(tx.T, degree)
test_matrix = build_poly(tX_test.T, degree)

w, _ = ridge_regression(y, train_matrix, lambda_)
w.shape
y_pred = predict_labels(w, tX_test.T)

In [ ]:
train_matrix.shape

### Least Square

In [ ]:
w, mse = least_squares(y, tx)
y_pred = predict_labels(w,tX_test)

### Least Square Stochastic Gradient Descent

In [6]:
initial_w = np.zeros(30)
max_iters = 100
gamma = 0.5

w, loss = least_squares_SGD(y, tx, initial_w, max_iters, gamma)
y_pred = predict_labels(w,tX_test)

### Results CSV

In [7]:
# Prepare final csv file with results
name = 'Predicteur'

create_csv_submission(ids_test, y_pred, name)